# Negative Sampling  : 네거티브 샘플링

### Embedding과 EmbeddingDot 계층

In [1]:
import numpy as np

# Embedding 계층
class Embedding :
    def __init__(self,W):
        self.params =[W]
        self.grads = [np.zeros_like(W)]
        self.idx = None
    
    # 순전파
    def forward(self,idx):
        W, = self.params
        self.idx = idx
        out = W[idx]
        return out
    
    # 역전파 
    def backward(self,dout):   
        dW, = self.grads
        dW[...] = 0
        np.add.at(dW, self.idx, dout)  

In [2]:
# EembeddingDot 계층
class EmbeddingDot:
    def __init__(self, W):
        self.embed = Embedding(W)
        self.params = self.embed.params
        self.grads = self.embed.grads
        self.cache = None

    def forward(self, h, idx):
        target_W = self.embed.forward(idx)
        out = np.sum(target_W * h, axis=1)

        self.cache = (h, target_W)
        return out

    def backward(self, dout):  
        h, target_W = self.cache
        dout = dout.reshape(dout.shape[0], 1) # 

        dtarget_W = dout * h
        self.embed.backward(dtarget_W)
        dh = dout * target_W
        return dh    


In [3]:
# EmbeddingDot 내의 forward 함수내에서 변수의 값 변화 정보
W = np.arange(21).reshape(7,3)
print('W:\n',W)

idx = np.array([[0,3,1]])
print('idx:\n',idx)

h = W[[0,1,2]]
print('h:\n',h)

embed = Embedding(W)
target_W = embed.forward(idx) # W에서 임베딩 처리
print('target_W:\n',target_W.reshape(-1,3))  # 2차원으로 변환
print('target_W*h:\n',target_W.reshape(-1,3)*h)  

out = np.sum(target_W.reshape(-1,3) * h, axis=1) # 수평방향 합
print('out:\n',out)

W:
 [[ 0  1  2]
 [ 3  4  5]
 [ 6  7  8]
 [ 9 10 11]
 [12 13 14]
 [15 16 17]
 [18 19 20]]
idx:
 [[0 3 1]]
h:
 [[0 1 2]
 [3 4 5]
 [6 7 8]]
target_W:
 [[ 0  1  2]
 [ 9 10 11]
 [ 3  4  5]]
target_W*h:
 [[ 0  1  4]
 [27 40 55]
 [18 28 40]]
out:
 [  5 122  86]


### 네거티브 샘플링 기법
https://ddiri01.tistory.com/310

In [8]:
import numpy as np

# 0에서 9가지의 숫자 중 하나를 무작위로 샘플링
np.random.choice(10)

6

In [10]:
np.random.choice(10)

8

In [34]:
# words에서 하나만 무작위로  샘플링
words = ['you','say','goodbye','i','hello','.']
np.random.choice(words)

'i'

In [35]:
# 5개만 무작위로 샘플링 (중복 허용)
np.random.choice(words,size=5)

array(['goodbye', 'you', 'goodbye', 'you', '.'], dtype='<U7')

In [36]:
# 5개만 무작위로 샘플링 (중복 금지)
np.random.choice(words,size=5,replace=False)

array(['goodbye', '.', 'i', 'you', 'hello'], dtype='<U7')

In [62]:
# 주어진 확률 분포에 따라 샘플링
p = [0.5,0.1,0.05,0.2,0.05,0.1]  # 합이 1.0
# p = [1.0,0.0,0.0,0.,0.,0.]  # 'you' 민 샘플링
print(sum(p))  # 1.000

np.random.choice(words,p=p)

1.0


'you'

In [65]:
# 네거티브 샘플링에서는 기본 확률 분포에 0.75를 제곱해준다
p = [0.7,0.29,0.01]
new_p = np.power(p,0.75) 
# 확률분포 값들에 모두 0.75제곱 처리 ==> 원래 확률이 낮은 단어를 버리지 않기 위해서
print(new_p)
new_p /= np.sum(new_p) # 다시 전체 요소의 합으로나우어 각 요소의 합이 1.0이 나오도록 새로운 확률 분포를 만든다
print(new_p)

[0.76528558 0.39518322 0.03162278]
[0.64196878 0.33150408 0.02652714]


In [66]:
print(2**4, np.power(2,4) ) # 2^4

16 16
